# Extending `pandas` to other ecosystems

In [23]:
    import pandas

In [42]:
    import sklearn.datasets, sklearn.discriminant_analysis, sklearn.model_selection, sklearn.metrics
    from toolz.curried import *; from toolz.curried.operator import *

Let's use the iris example.  _I'm sorry 😳._

In [43]:
    iris = sklearn.datasets.load_iris()

Create a tidy dataframe with testing and training indexes.

In [44]:
    df = pandas.DataFrame(
        iris['data'],
        pandas.CategoricalIndex(
            pandas.Index(iris['target']).map(
                iris['target_names'].__getitem__
            ), iris['target_names'], name='targets'),
        iris['feature_names']
    ).pipe(
        lambda df: pandas.concat(dict(zip(('train', 'test'), sklearn.model_selection.train_test_split(df))))
    )
    df.sample(2)

sepal length (cm)  sepal width (cm)  petal length (cm)  \
      targets                                                             
train virginica                6.7               3.1                5.6   
      setosa                   5.1               3.8                1.5   

                 petal width (cm)  
      targets                      
train virginica               2.4  
      setosa                  0.3

Let's use LDA.

In [45]:
    import sklearn.tree

In [46]:
    tree = sklearn.tree.DecisionTreeClassifier()
    lda = sklearn.discriminant_analysis.LinearDiscriminantAnalysis()

Train and predict the classes.

In [66]:
    from ipywidgets import interact

In [54]:
    df = df.pipe(
        do(lambda df: lda.fit(df.loc['train'], df.loc['train'].index))
    ).pipe(
        do(lambda df: tree.fit(df.loc['train'], df.loc['train'].index))
    ).pipe(
        lambda df: df
        .set_index(pandas.Index(tree.predict(df), name=tree), append=True)
        .set_index(pandas.Index(lda.predict(df), name=lda), append=True)
    )

In [59]:
    import IPython

In [61]:
    for model in (tree, lda):
        print(model)
        df.loc['test'].index.to_frame().pipe(
            lambda df: pandas.DataFrame(
                sklearn.metrics.confusion_matrix(df['targets'], df[model]),
                iris['target_names'], iris['target_names']
            )
        ).pipe(IPython.display.display)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


,setosa,versicolor,virginica
setosa,17,0,0
versicolor,0,10,1
virginica,0,1,9


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)


,setosa,versicolor,virginica
setosa,17,0,0
versicolor,0,10,1
virginica,0,0,10


In [72]:
    @interact
    def _(model=['tree','lda'], type=['test', 'train']):
        model = globals().get(model)
        df.loc[type].index.to_frame().pipe(
            lambda df: pandas.DataFrame(
                sklearn.metrics.confusion_matrix(df['targets'], df[model]),
                iris['target_names'], iris['target_names']
            )
        ).pipe(IPython.display.display)

interactive(children=(Dropdown(description='model', options=('tree', 'lda'), value='tree'), Dropdown(descripti…

In [8]:
    import ibis

    db = ibis.sqlite.connect('idiomatic_pandas.sqlite')

    db.list_tables()
    db.table('urls')['value'].execute()
    db.table('responses')['key'].execute()

C:\Anaconda3\lib\site-packages\ibis\sql\postgres\compiler.py:223: UserWarning: locale specific date formats (%c, %x, %X) are not yet implemented for Windows
  'for %s' % platform.system()


0    1e796e69aee7d5caaa8302de99522689d773d858bdc425...
1    ac00bfb030a3bb94b6bcb6b4e57cc7025d644a6673d15c...
2    f69c4ed7f42635e24d8b0d6e7cf4e15bd2edd8dc46c647...
Name: key, dtype: object